# Análisis de la Calidad de Vinos Rojos
Alan Badillo Salas (badillo.soft@hotmail.com)

Basado en el dataset de http://archive.ics.uci.edu/ml/datasets/Wine+Quality, vamos a evaluar la calidad del vino mediante factores medidos en el vino como son:

1. fixed acidity
2. volatile acidity
3. citric acid
4. residual sugar
5. chlorides
6. free sulfur dioxide
7. total sulfur dioxide
8. density
9. pH
10. sulphates
11. alcohol
12. quality (score between 0 and 10)

Dónde el último representa la calificación del 0 al 10 en la calidad del vino.

## Paso 1 - Adquisición de datos

Vamos a utilizar pandas para descargar el archivo `csv` con los datos mencionados anteriormente.

In [14]:
import pandas as pd

df = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv", sep=";")

print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
fixed acidity           1599 non-null float64
volatile acidity        1599 non-null float64
citric acid             1599 non-null float64
residual sugar          1599 non-null float64
chlorides               1599 non-null float64
free sulfur dioxide     1599 non-null float64
total sulfur dioxide    1599 non-null float64
density                 1599 non-null float64
pH                      1599 non-null float64
sulphates               1599 non-null float64
alcohol                 1599 non-null float64
quality                 1599 non-null int64
dtypes: float64(11), int64(1)
memory usage: 150.0 KB
None
   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04    

## Paso 2 - Preprocesamiento de datos

Todas las columnas son ordinales y cardinales (numéricas) por lo que no tenemos que hacer gran pre-procesamiento de datos, sólo vamos a ordenar los datos aleatoriamente para evitar pérdida de generalidad.

In [15]:
df = df.sample(frac=1)

print(df.head())

      fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
1403            7.2              0.33         0.33             1.7      0.061   
868             6.8              0.56         0.22             1.8      0.074   
559            13.0              0.47         0.49             4.3      0.085   
967             8.5              0.66         0.20             2.1      0.097   
106             7.8              0.41         0.68             1.7      0.467   

      free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
1403                  3.0                  13.0  0.99600  3.23       1.10   
868                  15.0                  24.0  0.99438  3.40       0.82   
559                   6.0                  47.0  1.00210  3.30       0.68   
967                  23.0                 113.0  0.99733  3.13       0.48   
106                  18.0                  69.0  0.99730  3.08       1.31   

      alcohol  quality  
1403     10.0        8  


## Paso 3 - Calcular el número de ejemplos de entrenamiento

Vamos a calcular la constante `k` que va a representar el 80% del total de muestras para tomarlas como parte del entrenamiento y el 20% restante usarlas para medir el desempeño del entrenamiento.

In [16]:
n = len(df)
k = int(0.8 * n)

print("Se tomarán {}/{} muestras para el entrenamiento".format(k, n))

Se tomarán 1279/1599 muestras para el entrenamiento


## Paso 4 - Seleccionar las columnas que serán utilizadas para el análisis

Vamos a seleccionar las columnas con las caracterísitcas para el entrenamiento (las muestras `x`) y las columnas objetivos para el entrenamiento (las muestras `y`).

In [24]:
x_columns = df.columns[0:11]
y_columns = df.columns[11:]

print(x_columns)
print(y_columns)

Index([u'fixed acidity', u'volatile acidity', u'citric acid',
       u'residual sugar', u'chlorides', u'free sulfur dioxide',
       u'total sulfur dioxide', u'density', u'pH', u'sulphates', u'alcohol'],
      dtype='object')
Index([u'quality'], dtype='object')


In [25]:
x = df.filter(items=x_columns)

y = df.filter(items=y_columns)

print(x.info())
print(y.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1599 entries, 1403 to 1224
Data columns (total 11 columns):
fixed acidity           1599 non-null float64
volatile acidity        1599 non-null float64
citric acid             1599 non-null float64
residual sugar          1599 non-null float64
chlorides               1599 non-null float64
free sulfur dioxide     1599 non-null float64
total sulfur dioxide    1599 non-null float64
density                 1599 non-null float64
pH                      1599 non-null float64
sulphates               1599 non-null float64
alcohol                 1599 non-null float64
dtypes: float64(11)
memory usage: 149.9 KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1599 entries, 1403 to 1224
Data columns (total 1 columns):
quality    1599 non-null int64
dtypes: int64(1)
memory usage: 25.0 KB
None


## Paso 5 - Seleccionar las muestras de entrenamiento y las de validación

Vamos a calcular `x_train`, `x_test`, `y_train` y `y_test`.

In [26]:
x_train = x[:k]
x_test  = x[k:]

y_train = y[:k]
y_test = y[k:]

print("Características de entrenamiento (x_train): {}".format(len(x_train)))
print("Características de validación (x_test): {}".format(len(x_test)))
print("Objetivos de entrenamiento (y_train): {}".format(len(y_train)))
print("Objetivos de validación (y_test): {}".format(len(y_test)))

Características de entrenamiento (x_train): 1279
Características de validación (x_test): 320
Objetivos de entrenamiento (y_train): 1279
Objetivos de validación (y_test): 320


## Paso 6 - Crear el modelo de la red neuronal (ANN)

Vamos a establecer una red neuronal artificial (ANN) con una capa densa de `22` unidades con activación `tanh` y otra capa densa de salida con `1` unidad con activación `relu`.

In [57]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()

model.add(Dense(44, activation="tanh", input_dim=11))
model.add(Dense(1, activation="relu"))

model.compile(optimizer="adam", loss="mse", metrics=["accuracy"])

print(model)

## Paso 7 - Entrenar el modelo

Vamos a entrenar a la red neuronal con `x_train` y `y_train`, en `5` epocas con bloques de `32` en `32`.

In [89]:
model.fit(x_train, y_train, epochs=5, batch_size=32)

Epoch 1/5
1279/1279 [==============================] - 0s 54us/step - loss: 0.3792 - acc: 0.6161
Epoch 2/5
1279/1279 [==============================] - 0s 52us/step - loss: 0.3772 - acc: 0.6200
Epoch 3/5
1279/1279 [==============================] - 0s 49us/step - loss: 0.3787 - acc: 0.6255
Epoch 4/5
1279/1279 [==============================] - 0s 51us/step - loss: 0.3826 - acc: 0.6208
Epoch 5/5
1279/1279 [==============================] - 0s 50us/step - loss: 0.3764 - acc: 0.6208


## Paso 8 - Validar el modelo

Vamos a evaluar el desempeño de la red neuronal mediante `x_test` y `y_test`.

In [90]:
metrics = model.evaluate(x_test, y_test)

print(metrics)

320/320 [==============================] - 0s 189us/step
[0.41524097323417664, 0.5875]


## Paso 9 - Contrastar los datos

Vamos a contrastastar los datos de validación contra los datos que arroja el modelo.

In [98]:
y_predict = model.predict(x_test)

for i in range(len(x_test)):
    xt = x_test.values[i]
    yt = y_test.values[i]
    yp = y_predict[i]
    print("{} <> {}".format(yt, yp))

[5] <> [5.7433095]
[5] <> [5.4500937]
[5] <> [5.2576194]
[5] <> [6.0433445]
[6] <> [5.462053]
[6] <> [6.353123]
[6] <> [5.410897]
[4] <> [5.726133]
[5] <> [5.8053803]
[6] <> [6.067354]
[7] <> [6.3026695]
[6] <> [5.47759]
[6] <> [5.28801]
[5] <> [5.2425094]
[6] <> [6.182472]
[3] <> [4.9315457]
[5] <> [4.9312487]
[5] <> [5.7630734]
[7] <> [6.503777]
[6] <> [6.015243]
[6] <> [6.052006]
[7] <> [5.85665]
[6] <> [5.718371]
[5] <> [5.5936155]
[6] <> [5.3778157]
[6] <> [5.693867]
[5] <> [5.464801]
[5] <> [6.185164]
[6] <> [5.4750276]
[5] <> [5.205648]
[5] <> [5.432565]
[5] <> [5.3762918]
[6] <> [5.761639]
[7] <> [5.964847]
[6] <> [5.3521543]
[6] <> [6.3394313]
[5] <> [4.874476]
[5] <> [5.433883]
[5] <> [5.0998464]
[6] <> [6.064123]
[7] <> [6.853362]
[5] <> [5.978847]
[5] <> [5.8933167]
[5] <> [5.320314]
[5] <> [5.3175664]
[5] <> [5.5400825]
[5] <> [5.2732325]
[7] <> [6.507204]
[6] <> [6.774592]
[5] <> [5.092625]
[5] <> [5.2964635]
[6] <> [6.150357]
[7] <> [6.6930957]
[5] <> [5.515864]
[5] <> [

## Paso 10 - Almacenar el modelo

Una vez que el modelo aprende correctamente a clasificar el vino (o al menos tiene buenos resultados), podemos almacenar el modelo y sus pesos utilizando el formato `JSON` para el modelo y el formato `HDF5` para los pesos como se muestra en este tutorial: https://machinelearningmastery.com/save-load-keras-deep-learning-models/.

In [102]:
model_json = model.to_json()

f = open("wine_model.json", "w")
f.write(model_json)
f.close()

model.save_weights("wine_weights.h5")

En futuro podemos cargar el modelo y los pesos desde sus archivos correspondientes sin tener que recrear ni la red neuronal ni el entrenamiento.

In [103]:
from keras.models import model_from_json

f = open("wine_model.json", "r")

model = model_from_json(f.read())

f.close()

model.load_weights("wine_weights.h5")

model.compile(optimizer="adam", loss="mse", metrics=["accuracy"])

## ... trabajar el modelo para predecir, evaluar o seguir entrenando

print(model.predict(np.array([
    [7.4, 0.70, 0.00, 1.9, 0.076, 11.0, 4.0, 0.9978, 3.51, 0.56, 9.4]
])))

[[3.2932699]]
